In [ ]:
import temet
# import h5py
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.cm as cm
import matplotlib.colors as colors
# import glob
import os
import random

# from scale_factor_helpers import *
# from spectra_visualization_helpers import *

import sys, os
# This is not super pretty, but I think this is the best way to import stuff from ../../../util?
CODE_ROOT = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
if CODE_ROOT not in sys.path:
    sys.path.insert(1, CODE_ROOT)

from util.sim_data_helpers import get_data_from_fof_folder
from util.cosmo_helpers import delta_z, corrected_z, redshift_wavelength_forward, redshift_wavelength_backward
from util.spectra_helpers import load_spectra_from_boxes

In [ ]:
gps_list = [i for i in range(50)]

base_path = "/vera/u/jerbo/my_ptmp/L25n256_suite"
gp_paths = [base_path+f"/gridpoint{i}/" for i in gps_list]

spectrum_num = 2796
wavelengths, fluxes, params = load_spectra_from_boxes(gp_paths, spectrum_num)

Ly_alpha_0 = 1215.67  # in Angström
z_box = 2
boxsize = 25  # in cMpc

Ly_alphas_z = []
corr_z_plot = []

for i in range(len(wavelengths)):
    # wavelength_current = wavelengths[i]
    (Omega_m, Omega_b, Omega_Lambda, h) = params[i]
    dz = delta_z(boxsize, z_box, Omega_m, Omega_Lambda, h)
    corr_z = corrected_z(boxsize, z_box, Omega_m, Omega_Lambda, h)
    corr_z_plot.append(corr_z)
    Ly_alphas_z.append(redshift_wavelength_forward(corr_z, Ly_alpha_0))

    difference = (temet.sim(gp_paths[i], redshift=2.0).dz - dz)/dz * 100
    if difference > 5:
        print("WARNING: Diff between temets dz and my dz is grater than 5%")
    # print(dz)
    # wavelengths[i] = redshift_wavelength_backward(corr_z, wavelength_current)


# print(Ly_alphas_z)


In [ ]:
fig, ax = plt.subplots(2,2)

for j in range(4):
    row = j//2
    col = j%2
    axe = ax[row][col]
    
    cosmo_par_index = j  # 0: Omega0, 1: OmegaBaryon, 2: OmegaLambda, 3: HubbleParam
    cosmo_par = [i[cosmo_par_index] for i in params]
    label_dir = {0: "Omega0", 1: "OmegaBaryon", 2: "OmegaLambda", 3: "HubbleParam"}

    axe.scatter(cosmo_par, corr_z_plot, marker="x")

    axe.set_ylabel("Scalefactor (corrected)")
    axe.set_xlabel(f"{label_dir[j]}")

plt.tight_layout()
plt.show()

In [ ]:
fig, axe = plt.subplots()

cosmo_par_index = 2  # 0: Omega0, 1: OmegaBaryon, 2: OmegaLambda, 3: HubbleParam
cosmo_par = [i[cosmo_par_index] for i in params]

norm = colors.Normalize(vmin=min(cosmo_par), vmax=max(cosmo_par))
# colormap = cm.viridis  # Choose any colormap: viridis, plasma, inferno, etc.
scalar_map = cm.ScalarMappable(norm=norm, cmap="jet")

for i in range(10, len(wavelengths)-30):
    color = scalar_map.to_rgba(cosmo_par[i])
    plt.plot(wavelengths[i], fluxes[i], alpha=0.6, c=color)

    plt.plot([Ly_alphas_z[i], Ly_alphas_z[i]],[0.3, 1.1], alpha=0.5, c=color, linestyle="--")


Ly_alpha_z_box = redshift_wavelength_forward(z_box, Ly_alpha_0)
plt.plot([Ly_alpha_z_box, Ly_alpha_z_box],[0.3, 1.1], c="black", linestyle="--")
plt.annotate(r"$Ly \alpha$", [Ly_alpha_0-1.5, 1.05], rotation=90)

# plt.annotate(r"$Ly \alpha$", [Ly_alpha_0-1.5, 1.05], rotation=90))

cbar = plt.colorbar(scalar_map, ax=axe)
cbar.set_label(r'$\Omega_\Lambda$')
plt.xlabel(r"Rest-frame wavelength [$\AA$]")
plt.ylabel(r"Relative flux ")
plt.xlim([3620, 3710])
plt.show()




In [ ]:
fig = plt.subplots()

for path in gp_paths:
    vel_disp = get_data_from_fof_folder(path, 3, "Subhalo", "SubhaloVelDisp")
    # vel_disp_disp = np.histogram(vel_disp)
    plt.hist(vel_disp, bins='auto', alpha=0.5)

plt.plot([1728, 1728], [0, 1400], c="black")
plt.annotate(r"Needed to explain 7 $\AA$ shift", [1700-100, 0+100], rotation=90)

# plt.xscale("log")
plt.title("Histogram of Velocity Dispersions of Halos in boxes")
plt.xlabel(r"$\sigma$ [km/s]")
plt.xlim([0, 2000])
plt.show()

In [ ]:
fig = plt.subplots()

for path in gp_paths:
    vel_disp = get_data_from_fof_folder(path, 3, "Subhalo", "SubhaloVel")
    # vel_disp_disp = np.histogram(vel_disp)
    vel_disp = [abs(i[2]) for i in vel_disp]
    plt.hist(vel_disp, bins='auto', alpha=0.5)

plt.plot([1728, 1728], [0, 1000], c="black")
plt.annotate(r"Needed to explain 7 $\AA$ shift", [1700-100, 0+100], rotation=90)

#plt.xscale("log")
plt.title("Histogram of Subhalo Velocities in boxes")
plt.xlabel(r"|subhalo velocity| [km/s]")
plt.xlim([0, 2000])
plt.show()

In [ ]:
# create the random spectra indices to use in the following
spectrum_numbers = random.sample(range(0, 10001), 12)
reference_path = "/vera/u/jerbo/my_ptmp/L25n256_suite/reference_point/"

In [ ]:
# note: the way this works is extremely inefficient at the moment. should change this if I want to run something similar repeatedly
fig, ax = plt.subplots(figsize=(20,20))

ax.set_xticks([])
ax.set_yticks([])
for spine in ax.spines.values():
    spine.set_visible(False)

gs = gridspec.GridSpec(4, 4, height_ratios=[1, 1, 1, 0.05])
axes = []

gps_list = [i for i in range(50)]

base_path = "/vera/u/jerbo/my_ptmp/L25n256_suite"
gp_paths = [base_path+f"/gridpoint{i}/" for i in gps_list]

Ly_alpha_0 = 1215.67  # in Angström
z_box = 2
boxsize = 25  # in cMpc

for j, spec_num in enumerate(spectrum_numbers): 

    print(j, spec_num)

    wavelengths, fluxes, params = load_spectra_from_boxes(gp_paths, spec_num)

    Ly_alphas_z = []

    for i in range(len(wavelengths)):
        wavelength_current = wavelengths[i]
        (Omega_m, Omega_b, Omega_Lambda, h) = params[i]
        dz = delta_z(boxsize, z_box, Omega_m, Omega_Lambda, h)
        corr_z = corrected_z(boxsize, z_box, Omega_m, Omega_Lambda, h)
        # print(dz)
        wavelengths[i] = redshift_wavelength_backward(corr_z, wavelength_current)
    
    row = j//4
    col = j%4
    
    ax = fig.add_subplot(gs[row, col])

    cosmo_par_index = 1  # 0: Omega0, 1: OmegaBaryon, 2: OmegaLambda, 3: HubbleParam
    cosmo_par = [i[cosmo_par_index] for i in params]

    norm = colors.Normalize(vmin=min(cosmo_par), vmax=max(cosmo_par))
    # colormap = cm.viridis  # Choose any colormap: viridis, plasma, inferno, etc.
    scalar_map = cm.ScalarMappable(norm=norm, cmap="jet")

    for i in range(len(wavelengths)):
        color = scalar_map.to_rgba(cosmo_par[i])
        ax.plot(wavelengths[i], fluxes[i], alpha=0.6, c=color)

    # ref_wavelengths, ref_fluxes, ref_params = load_reference(reference_path, spec_num)

    # ax.plot(ref_wavelengths, ref_fluxes, c="black")

    ax.plot([Ly_alpha_0, Ly_alpha_0],[-0.1, 1.1], c="black", linestyle="--")
    ax.annotate(r"$Ly \alpha$", [Ly_alpha_0-1.5, 1.05], rotation=90)

    ax.set_xlabel(r"Wavelength in $\AA$")
    ax.set_ylabel(r"Relative flux ")
    ax.set_xlim([1200, 1230])
    
    ax.set_title(f"Spectrum #{spec_num}")
    
    ax.set_xlabel("Rest-frame wavelength [$\AA$]")
    # ax.set_facecolor('black')
    axes.append(ax)


cbar_ax = fig.add_subplot(gs[3, :])
cbar = fig.colorbar(scalar_map, cax=cbar_ax, orientation='horizontal')
cbar.set_label(r'$\Omega_\text{b}$')
# plt.tight_layout(rect=[0, 0.05, 1, 1])
plt.savefig("plots/spectra_galery.pdf", format="PDF")
plt.show()

In [ ]:
# note: the way this works is extremely inefficient at the moment. should change this if I want to run something similar repeatedly
fig, ax = plt.subplots(figsize=(20,20))

ax.set_xticks([])
ax.set_yticks([])
for spine in ax.spines.values():
    spine.set_visible(False)

gs = gridspec.GridSpec(4, 4, height_ratios=[1, 1, 1, 0.05])
axes = []

gps_list = [i for i in range(50)]

base_path = "/vera/u/jerbo/my_ptmp/L25n256_suite"
gp_paths = [base_path+f"/gridpoint{i}/" for i in gps_list]

Ly_alpha_0 = 1215.67  # in Angström
z_box = 2
boxsize = 25  # in cMpc

for j, spec_num in enumerate(spectrum_numbers): 

    print(j, spec_num)

    wavelengths, fluxes, params = load_spectra_from_boxes(gp_paths, spec_num)

    Ly_alphas_z = []

    #for i in range(len(wavelengths)):
    #    wavelength_current = wavelengths[i]
    #    (Omega_m, Omega_b, Omega_Lambda, h) = params[i]
    #    dz = delta_z(boxsize, z_box, Omega_m, Omega_Lambda, h)
    #    corr_z = corrected_z(boxsize, z_box, Omega_m, Omega_Lambda, h)
    #    # print(dz)
    #    wavelengths[i] = redshift_wavelength_backward(corr_z, wavelength_current)
    
    row = j//4
    col = j%4
    
    ax = fig.add_subplot(gs[row, col])

    cosmo_par_index = 2  # 0: Omega0, 1: OmegaBaryon, 2: OmegaLambda, 3: HubbleParam
    cosmo_par = [i[cosmo_par_index] for i in params]

    norm = colors.Normalize(vmin=min(cosmo_par), vmax=max(cosmo_par))
    # colormap = cm.viridis  # Choose any colormap: viridis, plasma, inferno, etc.
    scalar_map = cm.ScalarMappable(norm=norm, cmap="jet")

    for i in range(len(wavelengths)):
        color = scalar_map.to_rgba(cosmo_par[i])
        ax.plot(wavelengths[i], fluxes[i], alpha=0.6, c=color)

    #ax.plot([Ly_alpha_0, Ly_alpha_0],[-0.1, 1.1], c="black", linestyle="--")
    #ax.annotate(r"$Ly \alpha$", [Ly_alpha_0-1.5, 1.05], rotation=90)

    ax.set_xlabel(r"Wavelength in $\AA$")
    ax.set_ylabel(r"Relative flux ")
    ax.set_xlim([3630, 3710])
    
    ax.set_title(f"Spectrum #{spec_num}")
    
    ax.set_xlabel("Observed Wavelength [$\AA$]")
    # ax.set_facecolor('black')
    axes.append(ax)


cbar_ax = fig.add_subplot(gs[3, :])
cbar = fig.colorbar(scalar_map, cax=cbar_ax, orientation='horizontal')
cbar.set_label(r'$\Omega_\Lambda$')
# plt.tight_layout(rect=[0, 0.05, 1, 1])
plt.savefig("plots/spectra_galery_no_correction_var.pdf", format="PDF")
plt.show()